In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from fake_useragent import UserAgent
from datetime import datetime
import time


In [11]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [12]:
local_time = datetime.now().strftime('%Y-%m-%d')

smtv_sports_url = "https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smtv_entertainment_url = "https://smtv.io/program/listwithgame/default?parent=44&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smtv_game_url = "https://smtv.io/program/listwithgame/default?parent=81&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_sports_url = "https://smzb.io/program/listwithgame/default?parent=1&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_entertainment_url = "https://smzb.io/program/listwithgame/default?parent=2&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_game_url = "https://smzb.io/program/listwithgame/default?parent=45&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
kl99_sports_url = "https://kl99.tv/program/listwithgame/default?parent=1&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
lj88_sports_url = "https://lj88.tv/program/listwithgame/default?parent=27&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y&from=gg"
ggzb_game_url = "https://ggzb.io/program/listwithgame/default?parent=1&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y&from=gg"



In [13]:
# 網址

# smtv 體育
# https://smtv.io/program/43
# smtv 娛樂
# https://smtv.io/program/44
# smtv 電競
# https://smtv.io/program/81

# ------------------------

# smzb 體育
# https://smzb.io/program/1
# smzb 娛樂
# https://smzb.io/program/2
# smzb 電競    
# https://smzb.io/program/45

# ------------------------

# ggzb 電競
# https://ggzb.io/

# ------------------------

# lj88 體育
# https://lj88.tv/

# -----------------------------------------------------------------------


# 比較列表

# smtv體育 diff smzb體育
# smtv體育 diff kl99體育
# smtv體育 diff lj88體育

# ------------------------

# smtv電競 diff smzb電競
# smtv電競 diff ggzb電競

# ------------------------

# smtv娛樂 diff smzb娛樂


In [14]:
def Get_Schedule_to_DF(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]
            account = []

            for k in schedule["result"][i]["data"][j]["account"]:
                nickname = k["nickname"]
                live_status = k["live_status"]

                if live_status == 1:
                    status = "開播"
                else:
                    status = "未開播"
                account.append({"nickname":nickname,"live_status":status})

            dict_={
                "live_types_name":live_types_name,
                "live_types_child_name":live_types_child_name,
                "lname":lname,
                "time":time,
                "hname":hname,
                "aname":aname,
                "account":account
            }
            result.append(dict_)

    df = pd.DataFrame(result)

    return df


In [15]:
# 含信號源
def Get_Schedule_diff_signal(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]
            account = []

            for k in schedule["result"][i]["data"][j]["account"]:
                nickname = k["nickname"]
                live_status = k["live_status"]

                if live_status == 1:
                    status = "開播"
                else:
                    status = "未開播"
                account.append({"nickname":nickname,"live_status":status})

            result_str = time + "  " + lname + "  " + hname + " vs " + aname + "  " + str(account)


            result.append(result_str)

    return result

In [16]:
# 不含信號源
def Get_Schedule_diff(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]

            result_str = time + "  " + lname + "  " + hname + " vs " + aname

            result.append(result_str)

    return result


In [17]:
def Schedule_diff(diff_A,diff_B):
    diff_1 = set(diff_A)
    diff_2 = set(diff_B)
    diff_result = sorted(list(diff_1.difference(diff_2)))
    return diff_result

In [20]:
# 比較賽事差異

print("----------以下為體育賽事----------")

smtv_sports = Get_Schedule_diff(smtv_sports_url)
smzb_sports = Get_Schedule_diff(smzb_sports_url)
diff_result = Schedule_diff(smtv_sports,smzb_sports)
print("分類:體育，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
# print("==============================================")

# smtv_sports = Get_Schedule_diff(smtv_sports_url)
# kl99_sports = Get_Schedule_diff(kl99_sports_url)
# diff_result = Schedule_diff(smtv_sports,kl99_sports)
# print("分類:體育，smtv diff kl99")
# print("數量:",len(diff_result))
# for i in diff_result:
#     print(i)
    
print("==============================================")

smtv_sports = Get_Schedule_diff(smtv_sports_url)
lj88_sports = Get_Schedule_diff(lj88_sports_url)
diff_result = Schedule_diff(smtv_sports,lj88_sports)
print("分類:體育，smtv diff lj88")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("----------以下為娛樂直播----------")

smtv_entertainment = Get_Schedule_diff(smtv_entertainment_url)
smzb_entertainment = Get_Schedule_diff(smzb_entertainment_url)
diff_result = Schedule_diff(smtv_entertainment,smzb_entertainment)
print("分類:娛樂，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("----------以下為電競賽事----------")

smtv_game = Get_Schedule_diff(smtv_game_url)
smzb_game = Get_Schedule_diff(smzb_game_url)
diff_result = Schedule_diff(smtv_game,smzb_game)
print("分類:電競，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("==============================================")

smtv_game = Get_Schedule_diff(smtv_game_url)
ggzb_game = Get_Schedule_diff(ggzb_game_url)
diff_result = Schedule_diff(smtv_game,ggzb_game)
print("分類:電競，smtv diff ggzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    




----------以下為體育賽事----------
分類:體育，smtv diff smzb
數量: 1
2020-06-25 01:00:00  立陶甲  班加 vs 瑞特瑞埃
分類:體育，smtv diff lj88
數量: 37
2020-06-24 08:00:00  虚拟赛事  北韩 vs 科威特
2020-06-24 10:00:00  虚拟赛事  日本 vs 柬埔寨
2020-06-24 12:00:00  虚拟赛事  越南 vs 澳洲
2020-06-24 14:00:00  虚拟赛事  菲律宾 vs 土库曼
2020-06-24 16:00:00  虚拟赛事  苏联 vs 波兰
2020-06-24 16:30:00  国际友谊  布拉格斯拉维亚B队 vs 波希米亚人1905B队
2020-06-24 20:00:00  虚拟赛事  巴西 vs 瑞典
2020-06-24 22:00:00  虚拟赛事  德国 vs 奥地利
2020-06-24 22:45:00  科索超  德利塔格尼拉內 vs 科索沃乌西特里
2020-06-24 22:45:00  科索超  杜卡吉尼 vs KF格尼拉内
2020-06-24 22:45:00  科索超  特雷普查89 vs 法拿莫达利(KOS)
2020-06-24 22:55:00  匈甲  梅索科菲德 vs 德布勒森尼
2020-06-24 23:00:00  乌克甲  奥博隆 vs 艾维铵哈德
2020-06-24 23:00:00  国际友谊  塔波斯科 vs 贝内绍夫
2020-06-24 23:00:00  国际友谊  塞那乔其 vs 洛瓦涅米
2020-06-24 23:00:00  国际友谊  皮斯克 vs 皮耳森B队
2020-06-24 23:00:00  白俄甲  泽辛斯克兵工厂 vs FK戈梅利火车头
2020-06-24 23:30:00  国际友谊  乌尼邱夫 vs 布兰斯科
2020-06-25 00:00:00  斯洛伐克杯  鲁森比洛克 vs 斯特里达
2020-06-25 01:00:00  丹麦乙  AB格莱萨克瑟 vs 艾华塔
2020-06-25 01:00:00  丹麦乙  VSK奥胡斯 vs 希勒罗德
2020-06-25 01:00:00  丹麦乙  塞维斯